# 1整理数据

In [1]:
# Load the packages
library(haven)
library(dplyr)
library(tidyverse)


Attaching package: ‘dplyr’




The following objects are masked from ‘package:stats’:

    filter, lag




The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.5.1     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1


── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# 读取 CSV 文件并命名为 data
data <- read.csv("/kaggle/input/iatdata-weighted/survey_data_weighted_filtered.csv")
data <- data%>%
  mutate(country_iso = ifelse(country_iso == "UK", "GB", country_iso)) 
# 查看前几行以确认导入成功
glimpse(data)
head(data)


Rows: 2,236,528
Columns: 46
$ session_id                <dbl> -2999930, -2999920, -2999915, -2999892, -299…
$ age                       <int> 19, 30, 35, 24, 26, 28, 43, 18, 19, 38, 21, …
$ birthyear                 <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ year                      <int> 2005, 2005, 2005, 2005, 2005, 2005, 2005, 20…
$ sex                       <chr> "f", "f", "m", "f", "n", "m", "m", "f", "m",…
$ birthsex                  <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ genderIdentity_0002       <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ transIdentity             <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ att_5                     <int> 3, 3, 3, 4, 3, 4, 3, 3, 4, 4, 3, 4, 4, 3, 3,…
$ att_7                     <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ D_biep.Young_Good_all     <dbl> 0.49379792, 0.11875695, 0.61289950, 1.047096…
$ PCT_error_3467            <dbl> 5.0000000, 6.6666667, 5.8333333, 5.8333333, …
$ pct_400   

,session_id,age,birthyear,year,sex,birthsex,genderIdentity_0002,transIdentity,att_5,att_7,⋯,genderIdentity_0002_clean,Sex,att_7_scaled,Att,country_iso,country_source,lable1,lable2,AgeGroup,post_weight
,<dbl>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,-2999930,19,NA,2005,f,NA,NA,NA,3,NA,⋯,NA,2,NA,3,US,countrycit,NA,NA,15-19,0.2816792
2,-2999920,30,NA,2005,f,NA,NA,NA,3,NA,⋯,NA,2,NA,3,US,countrycit,NA,NA,30-34,0.7606034
3,-2999915,35,NA,2005,m,NA,NA,NA,3,NA,⋯,NA,1,NA,3,US,countrycit,NA,NA,35-39,1.5000306
4,-2999892,24,NA,2005,f,NA,NA,NA,4,NA,⋯,NA,2,NA,4,US,countrycit,NA,NA,20-24,0.2940237
5,-2999885,26,NA,2005,n,NA,NA,NA,3,NA,⋯,NA,1,NA,3,US,countrycit,NA,NA,25-29,0.8953045
6,-2999882,28,NA,2005,m,NA,NA,NA,4,NA,⋯,NA,1,NA,4,US,countrycit,NA,NA,25-29,0.8953045


In [3]:
data$iat <- data$D_biep.Young_Good_all

In [4]:
summary(data$iat)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.9030  0.2016  0.4619  0.4442  0.7055  1.8595 

In [5]:
table(data$country_iso)


     AF      AR      AT      AU      BE      BR      CA      CC      CH      CN 
   2132    2737    7548   25556    3077    6813   54879    2070    4873   13721 
     CO      DE      DK      FI      FR      GB      GF      GH      HK      ID 
   4440   14881    2651    3249    7703   18834    2060    2728    3393    2014 
     IE      IN      IR      IT      JP      KR      KY      LK      MX      MY 
   6903   23369    8103    6300    3795    5666    6868    4180   12455    2932 
     NC      NE      NL      nu      NZ      PH      PL      PT      QA      RO 
   5573    3357    3286    7217    3658    6645    4197    2279    2053    3416 
     RU      SE      SG      TJ      TK      UG      UM      US      VG      WF 
   3721    2843    5212    2266    4597 1199884   47568  647430    2545    8490 
     ZA 
   4361 

# 2 全球分性别iat_lexis

In [6]:
# 载入 dplyr 包
library(dplyr)

# 提取关键列
global_data <- data[, c("Sex", "Period", "Age", "iat")]

# 男性子集
global_data_m <- subset(global_data, Sex == 1)[, c("Period", "Age", "iat")]

# 女性子集
global_data_f <- subset(global_data, Sex == 2)[, c("Period", "Age", "iat")]

# 汇总男性数据
pivot_m <- global_data_m %>%
  group_by(Period, Age) %>%
  summarise(mean_iat = mean(iat, na.rm = TRUE)) %>%
  arrange(Period, Age)

# 汇总女性数据
pivot_f <- global_data_f %>%
  group_by(Period, Age) %>%
  summarise(mean_iat = mean(iat, na.rm = TRUE)) %>%
  arrange(Period, Age)


`summarise()` has grouped output by 'Period'. You can override using the
`.groups` argument.


`summarise()` has grouped output by 'Period'. You can override using the
`.groups` argument.


In [7]:
glimpse(pivot_f)
head(pivot_f)
summary(pivot_f$mean_iat)

Rows: 1,586
Columns: 3
Groups: Period [23]
$ Period   <int> 2002, 2003, 2003, 2003, 2003, 2003, 2003, 2003, 2003, 2003, 2…
$ Age      <int> 55, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2…
$ mean_iat <dbl> 0.01514685, 0.41114627, 0.41988731, 0.45778365, 0.46888733, 0…


Period,Age,mean_iat
<int>,<int>,<dbl>
2002,55,0.01514685
2003,15,0.41114627
2003,16,0.41988731
2003,17,0.45778365
2003,18,0.46888733
2003,19,0.48493757


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-0.5006  0.4042  0.4272  0.4353  0.4524  1.2226 

In [8]:
glimpse(pivot_m)
head(pivot_m)
summary(pivot_m$mean_iat)

Rows: 1,569
Columns: 3
Groups: Period [22]
$ Period   <int> 2003, 2003, 2003, 2003, 2003, 2003, 2003, 2003, 2003, 2003, 2…
$ Age      <int> 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3…
$ mean_iat <dbl> 0.5570952, 0.4928653, 0.5644925, 0.5361579, 0.5686035, 0.5502…


Period,Age,mean_iat
<int>,<int>,<dbl>
2003,15,0.5570952
2003,16,0.4928653
2003,17,0.5644925
2003,18,0.5361579
2003,19,0.5686035
2003,20,0.5502688


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-0.1791  0.4753  0.5023  0.5155  0.5371  1.3020 

In [9]:
# 男性数据的离散标签（7 类）
pivot_m$iat_disc <- cut(
  pivot_m$mean_iat,
  breaks = c(-Inf, -0.75, -0.5, -0.1, 0.1, 0.5, 0.75, Inf),
  labels = c(
    "极强偏好老年人",
    "明显偏好老年人",
    "轻微偏好老年人",
    "中性",
    "轻微偏好年轻人",
    "明显偏好年轻人",
    "极强偏好年轻人"
  ),
  right = TRUE,
  include.lowest = TRUE
)

# 女性数据的离散标签（7 类）
pivot_f$iat_disc <- cut(
  pivot_f$mean_iat,
  breaks = c(-Inf, -0.75, -0.5, -0.1, 0.1, 0.5, 0.75, Inf),
  labels = c(
    "极强偏好老年人",
    "明显偏好老年人",
    "轻微偏好老年人",
    "中性",
    "轻微偏好年轻人",
    "明显偏好年轻人",
    "极强偏好年轻人"
  ),
  right = TRUE,
  include.lowest = TRUE
)

# ✅ 强制设置 factor 顺序
levels_7 <- c(
  "极强偏好年轻人",
  "明显偏好年轻人",
  "轻微偏好年轻人",
  "中性",
  "轻微偏好老年人",
  "明显偏好老年人",
  "极强偏好老年人"
)
# ✅ 手动定义7个分类对应的颜色（红→蓝发散色调）
colors_7 <- c(
  "极强偏好年轻人" = "#2166AC",
  "明显偏好年轻人" = "#4393C3",
  "轻微偏好年轻人" = "#92C5DE",
  "中性"           = "#F7F7F7",
  "轻微偏好老年人" = "#F4A582",
  "明显偏好老年人" = "#D6604D",
  "极强偏好老年人" = "#B2182B"
)




In [10]:
# 第一部分：初始化环境 ---------------------------------------------------
library(ggplot2)  # 用于数据可视化
library(dplyr)    # 用于数据处理和管道操作
library(tidyr)    # 用于数据重塑(长宽格式转换)
library(readr)    # 用于高效读写CSV文件
library(sysfonts) # 用于字体管理
library(showtextdb) # 用于字体数据库
library(showtext)   # 用于在R图形中使用系统字体
showtext.auto()   # 激活showtext功能，允许使用系统字体


'showtext.auto()' is now renamed to 'showtext_auto()'
The old version still works, but consider using the new function in future code



In [11]:
library(ggplot2)
pivot_m$iat_disc <- factor(pivot_m$iat_disc, levels = levels_7)

plot_lexis_m <- ggplot(pivot_m) +
  # 热图
  geom_raster(aes(x = Period + 0.5, y = Age + 0.5, fill = iat_disc)) +

  # Lexis 网格线
  geom_hline(yintercept = seq(15, 89, 5), alpha = 0.2, lty = "dotted") +
  geom_vline(xintercept = seq(2003, 2024, 7), alpha = 0.2, lty = "dotted") +  # ✅ 修改：年份线
  geom_abline(intercept = seq(-100, 100, 10) - 2003, alpha = 0.2, lty = "dotted") +  # ✅ 关键同步：起点年份是 2003

  # 色标
  scale_fill_manual(
  name = NULL,
  values = colors_7,drop=FALSE
)+

  # 坐标轴设置
  scale_x_continuous(
    name = "年份",
    breaks = seq(2003, 2024, 7),     # ✅ 修改：2年为步长
    limits = c(2003, 2024),
    expand = c(0, 0)
  ) +
  scale_y_continuous("年龄", expand = c(0, 0)) +

  # 比例设定（可保留 coord_equal，也可以换成 coord_fixed(ratio = 0.5)）
  coord_equal() +

  guides(fill = guide_legend(reverse = TRUE)) +
  theme_void() +
  theme(
    axis.text = element_text(color = "black"),
    axis.text.y = element_text(),
    axis.text.x = element_text()
  )

# 保存图形
ggsave(
  filename = "/kaggle/working/iat_lexis_male.tiff",
  plot = plot_lexis_m,
  dpi = 600
)


Saving 12.5 x 6.67 in image


Warning message:
“Removed 74 rows containing missing values or values outside the scale range
(`geom_raster()`).”


In [12]:
library(ggplot2)
pivot_f$iat_disc <- factor(pivot_f$iat_disc, levels = levels_7)

plot_lexis_f <- ggplot(pivot_f) +
  # 热图
  geom_raster(aes(x = Period + 0.5, y = Age + 0.5, fill = iat_disc)) +

  # Lexis 网格线
  geom_hline(yintercept = seq(15, 89, 5), alpha = 0.2, lty = "dotted") +
  geom_vline(xintercept = seq(2003, 2024, 7), alpha = 0.2, lty = "dotted") +  # ✅ 修改：年份线
  geom_abline(intercept = seq(-100, 100, 10) - 2003, alpha = 0.2, lty = "dotted") +  # ✅ 关键同步：起点年份是 2003

  # 色标
  scale_fill_manual(
  name = NULL,
  values = colors_7,drop=FALSE
)+

  # 坐标轴设置
  scale_x_continuous(
    name = "年份",
    breaks = seq(2003, 2024, 7),     # ✅ 修改：2年为步长
    limits = c(2003, 2024),
    expand = c(0, 0)
  ) +
  scale_y_continuous("年龄", expand = c(0, 0)) +

  # 比例设定（可保留 coord_equal，也可以换成 coord_fixed(ratio = 0.5)）
  coord_equal() +

  guides(fill = guide_legend(reverse = TRUE)) +
  theme_void() +
  theme(
    axis.text = element_text(color = "black"),
    axis.text.y = element_text(),
    axis.text.x = element_text()
  )

# 保存图形
ggsave(
  filename = "/kaggle/working/iat_lexis_famale.tiff",
  plot = plot_lexis_f,
  dpi = 600
)


Saving 12.5 x 6.67 in image


Warning message:
“Removed 75 rows containing missing values or values outside the scale range
(`geom_raster()`).”


# 3 中国分性别 iat_lexis

In [13]:
# 提取中国的数据（country_iso == "CN"）并选取关键列
CN_data <- subset(data, country_iso == "CN")[, c("Sex", "Period", "Age", "iat")]
# 男性子集
CN_data_m <- subset(CN_data, Sex == 1)[, c("Period", "Age", "iat")]
# 女性子集
CN_data_f <- subset(CN_data, Sex == 2)[, c("Period", "Age", "iat")]

# 汇总男性数据
pivot_m <- CN_data_m %>%
  group_by(Period, Age) %>%
  summarise(mean_iat = mean(iat, na.rm = TRUE)) %>%
  arrange(Period, Age)

# 汇总女性数据
pivot_f <- CN_data_f %>%
  group_by(Period, Age) %>%
  summarise(mean_iat = mean(iat, na.rm = TRUE)) %>%
  arrange(Period, Age)


`summarise()` has grouped output by 'Period'. You can override using the
`.groups` argument.


`summarise()` has grouped output by 'Period'. You can override using the
`.groups` argument.


In [14]:
# 男性数据的离散标签（7 类）
pivot_m$iat_disc <- cut(
  pivot_m$mean_iat,
  breaks = c(-Inf, -0.75, -0.5, -0.1, 0.1, 0.5, 0.75, Inf),
  labels = c(
    "极强偏好老年人",
    "明显偏好老年人",
    "轻微偏好老年人",
    "中性",
    "轻微偏好年轻人",
    "明显偏好年轻人",
    "极强偏好年轻人"
  ),
  right = TRUE,
  include.lowest = TRUE
)

# 女性数据的离散标签（7 类）
pivot_f$iat_disc <- cut(
  pivot_f$mean_iat,
  breaks = c(-Inf, -0.75, -0.5, -0.1, 0.1, 0.5, 0.75, Inf),
  labels = c(
    "极强偏好老年人",
    "明显偏好老年人",
    "轻微偏好老年人",
    "中性",
    "轻微偏好年轻人",
    "明显偏好年轻人",
    "极强偏好年轻人"
  ),
  right = TRUE,
  include.lowest = TRUE
)

# ✅ 强制设置 factor 顺序
levels_7 <- c(
  "极强偏好年轻人",
  "明显偏好年轻人",
  "轻微偏好年轻人",
  "中性",
  "轻微偏好老年人",
  "明显偏好老年人",
  "极强偏好老年人"
)


# 建议颜色（红-灰-蓝的 7 色渐变）
colors_7 <- c(
  "极强偏好年轻人" = "#2166AC",
  "明显偏好年轻人" = "#4393C3",
  "轻微偏好年轻人" = "#92C5DE",
  "中性"           = "#F7F7F7",
  "轻微偏好老年人" = "#F4A582",
  "明显偏好老年人" = "#D6604D",
  "极强偏好老年人" = "#B2182B"
)


In [15]:
library(ggplot2)
pivot_m$iat_disc <- factor(pivot_m$iat_disc, levels = levels_7)

plot_lexis_m <- ggplot(pivot_m) +
  # 热图
  geom_raster(aes(x = Period + 0.5, y = Age + 0.5, fill = iat_disc)) +

  # Lexis 网格线
  geom_hline(yintercept = seq(15, 89, 5), alpha = 0.2, lty = "dotted") +
  geom_vline(xintercept = seq(2003, 2024, 7), alpha = 0.2, lty = "dotted") +  # ✅ 修改：年份线
  geom_abline(intercept = seq(-100, 100, 10) - 2003, alpha = 0.2, lty = "dotted") +  # ✅ 关键同步：起点年份是 2003

  # 色标
 scale_fill_manual(
  name = NULL,
  values = colors_7,drop=FALSE
)+
  # 坐标轴设置
  scale_x_continuous(
    name = "年份",
    breaks = seq(2003, 2024, 7),     # ✅ 修改：2年为步长
    limits = c(2003, 2024),
    expand = c(0, 0)
  ) +
  scale_y_continuous("年龄", expand = c(0, 0)) +

  # 比例设定（可保留 coord_equal，也可以换成 coord_fixed(ratio = 0.5)）
  coord_equal() +

  guides(fill = guide_legend(reverse = TRUE)) +
  theme_void() +
  theme(
    axis.text = element_text(color = "black"),
    axis.text.y = element_text(),
    axis.text.x = element_text()
  )

# 保存图形
ggsave(
  filename = "/kaggle/working/CN_male.tiff",
  plot = plot_lexis_m,
  dpi = 300
)


Saving 12.5 x 6.67 in image


Warning message:
“Raster pixels are placed at uneven horizontal intervals and will be shifted
ℹ Consider using `geom_tile()` instead.”


In [16]:
library(ggplot2)
pivot_f$iat_disc <- factor(pivot_f$iat_disc, levels = levels_7)
plot_lexis_f <- ggplot(pivot_f) +
  # 热图
  geom_raster(aes(x = Period + 0.5, y = Age + 0.5, fill = iat_disc)) +

  # Lexis 网格线
  geom_hline(yintercept = seq(15, 89, 5), alpha = 0.2, lty = "dotted") +
  geom_vline(xintercept = seq(2003, 2024, 2), alpha = 0.2, lty = "dotted") +  # ✅ 修改：年份线
  geom_abline(intercept = seq(-100, 100, 10) - 2003, alpha = 0.2, lty = "dotted") +  # ✅ 关键同步：起点年份是 2003

  # 色标
scale_fill_manual(
  name = NULL,
  values = colors_7,drop=FALSE
)+

  # 坐标轴设置
  scale_x_continuous(
    name = "年份",
    breaks = seq(2003, 2024, 7),     # ✅ 修改：2年为步长
    limits = c(2003, 2024),
    expand = c(0, 0)
  ) +
  scale_y_continuous("年龄", expand = c(0, 0)) +

  # 比例设定（可保留 coord_equal，也可以换成 coord_fixed(ratio = 0.5)）
  coord_equal() +

  guides(fill = guide_legend(reverse = TRUE)) +
  theme_void() +
  theme(
    axis.text = element_text(color = "black"),
    axis.text.y = element_text(),
    axis.text.x = element_text()
  )

# 保存图形
ggsave(
  filename = "/kaggle/working/CN_famale.tiff",
  plot = plot_lexis_f,
  dpi = 300
)


Saving 12.5 x 6.67 in image


Warning message:
“Raster pixels are placed at uneven horizontal intervals and will be shifted
ℹ Consider using `geom_tile()` instead.”


Warning message:
“Removed 2 rows containing missing values or values outside the scale range
(`geom_raster()`).”
